### 叔叔我啊，要开始训练了捏

首先，获取模型和数据


In [ ]:
import numpy as np
import torch
import os
from tqdm import tqdm

from dataset import MyDataset
from model import MyTransformerEncoder

device = "cuda" if torch.cuda.is_available() else "cpu"

model = MyTransformerEncoder(d_model=100, output_dimension=1)
train_data = np.load("./npys/train/sentence_code.npy")
train_labels = np.load("./npys/train/labels.npy")
vocabulary_vectors = np.load("./npys/vocabulary_vectors.npy")
dataset = MyDataset(train_data[:500], train_labels[:500], vocabulary_vectors)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, num_workers=0) # Windows上，num_workers太高反而满，很sb


optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def train(max_epoch):
    loss_fn = torch.nn.functional.cross_entropy
    for epoch in range(max_epoch):
        avg_loss = []
        for X, y in tqdm(dataloader):
            output = model(X)
            loss = loss_fn(output, y)
            avg_loss.append(loss.item())
            loss.backward()
            optimizer.step()
        print("Epoch: %d | loss: %f"%(epoch, np.average(avg_loss)))

In [ ]:
train(100)
os.makedirs("./trained_models/", exist_ok=True)
torch.save(model, "./trained_models/transformer%d"%100)